Constructing Features from Scratch

In [4]:
#import libraries

import os
import sys
import numpy as np
import torchtext
import pandas as pd
import re
import requests
import pickle
import nltk
from nltk.corpus import wordnet as wn
nltk.download('popular')

from lib import data_utils, preprocess

#get path
src_path = os.getcwd()

datapath = src_path + '/Data/'
PLcopath = datapath + 'AnnotatedPLData/PLCoref'
PLpath = datapath + 'AnnotatedPLData/PLTexts'
CENcopath = datapath + 'AnnotatedCENData/CENCoref'
CENpath = datapath + 'AnnotatedCENData/CENTexts'
ONpath = datapath + 'AnnotatedONData/ONCoref'

PLInter = src_path + '/output/IntermediateFilesPL/'
CENInter = src_path + '/output/IntermediateFilesCEN/'
ONInter = src_path + '/output/IntermediateFilesON/'

#import data

#create empty data frame
PL_data = pd.DataFrame(columns=['corpusID', 'character', 'animacy', 'coref_chain', 'chain_head', 'head_of_head', 'chain_len', 'CL'])

#append all PL texts and features into one dataframe
for n in range(1,47):
# for n in range(1,2):

    print(n)

    #get story path
    storycopath = PLcopath +'/story' + str(n) + '.txt'
    storypath = PLpath + '/story' + str(n) + '.txt'
    storyid = 'story'+ str(n)

    #read in story
    corpus = data_utils.read_story(storycopath)

    # read in from intermediate files
    # list of features
    # features = ["CN", "Dep", "NER", "SS", "Triple", "WN"]

    # for f in features:
    #     #empty list
    #     feat = []
    #     with open(PLInter + f + 'FeatureBoolean'+'/Story' + str(n) + '.txt', 'r') as doc:
    #         for line in doc:
    #             feat.append(eval(line.rstrip()))
    
    #     corpus[f] = feat

    #get ss feature
    sslist = preprocess.semantic_subj(storypath)
    #remove leading The/A's in the sematic list
    sslist = [re.sub('^(The |A )','',s, flags=re.IGNORECASE) for s in sslist]
    pattern = '|'.join(sslist)
    pattern = pattern.replace('?|','')
    pattern = pattern.replace('!|','')
    pattern = pattern.replace('.|','')
    pattern = pattern.replace('(|','')
    pattern = pattern.replace('(','')
    pattern = pattern.replace(')','')
    
    #create binary flag variable for ss feat
    corpus['SS'] = corpus['head_of_head'].str.contains(pattern)
    corpus['SS'] = corpus['SS'].replace({True:1, False:0})

    #get ner feature
    nerlist = preprocess.ner_person(storypath)
    pattern = '|'.join(nerlist)
    pattern = pattern.replace('(','')
    pattern = pattern.replace(')','')
    pattern = pattern.replace('[','')
    pattern = pattern.replace(']','')

    #create binary flag variable for ss feat
    corpus['NER'] = corpus['head_of_head'].str.contains(pattern)
    corpus['NER'] = corpus['NER'].replace({True:1, False:0})


    #create binary flag variable for wn feat
    #get wordnet synset of head of chain
    wn_input = corpus['head_of_head'].apply(lambda word: pd.Series(wn.synsets(word)))
   
    #fill blanks with unrelated word to person
    wn_input[0] = wn_input[0].fillna(wn.synset('strong.a.01'))
   
    # get common synonym with person
    per = wn.synset('person.n.01')
    test = wn_input[0].apply(lambda syn: pd.Series(syn.lowest_common_hypernyms(per)))
   
    # test if head of chain related to person
    corpus['WN']= test[0]==per
    corpus['WN'] = corpus['WN'].replace({True:1, False:0})

    #get dp feat
    dplist = preprocess.dep_link(storypath)
    dplist = list(set(dplist))
    pattern='|'.join(dplist)
    #remove punct
    pattern = pattern.replace('?|','')
    pattern = pattern.replace('!|','')
    pattern = pattern.replace('.|','')
    pattern = pattern.replace('(|','')
    pattern = pattern.replace(')|','')
    pattern = pattern.replace('(','')
    pattern = pattern.replace(')','')

    #create binary flag variable for dp feat
    corpus['DP'] = corpus['head_of_head'].str.contains(pattern)
    corpus['DP'] = corpus['DP'].replace({True:1, False:0})

    #get triple feat
    tplist = preprocess.triple(storypath)
    tplist = list(set(tplist))
    #remove leading The/A's in the sematic list
    tplist = [re.sub('^(The |A )','',s, flags=re.IGNORECASE) for s in tplist]
    pattern='|'.join(tplist)
    pattern = pattern.replace('?|','')
    pattern = pattern.replace('!|','')
    pattern = pattern.replace('.|','')
    pattern = pattern.replace('(|','')
    pattern = pattern.replace(')|','')
    pattern = pattern.replace('(','')
    pattern = pattern.replace(')','')

    #create binary flag variable for ss feat
    corpus['TP'] = corpus['head_of_head'].str.contains(pattern)
    corpus['TP'] = corpus['TP'].replace({True:1, False:0})

    # get conceptnet feat
    urlreq = 'https://api.conceptnet.io/c/en/'+corpus['head_of_head']

    #default no presence of person mentioned
    corpus['CN'] = 0

    for i in range(len(urlreq)):

        #make request to concept net api
        response = requests.get(urlreq[i])
        obj = response.json()
        #get list of edges
        cnlist = [edge['@id'] for edge in obj['edges']]

        #if person is in list then flag
        if any('person' in s for s in cnlist):
            val = 1
            corpus['CN'][i]=val
    
    #create feature for freq of head of chain term in text
    tf_dict = preprocess.term_freq(storypath)
    corpus['TF'] = corpus['head_of_head'].map(tf_dict)
    
    #append to dataframe
    PL_data = pd.concat([PL_data, corpus], ignore_index=True)

PL_data

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/eileen/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/eileen/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/eileen/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/eileen/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/eileen/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/eileen/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to

1


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/tmp/ipykernel_268959/1917952585.py:93: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specif

2


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/tmp/ipykernel_268959/1917952585.py:100: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Speci

...

45


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/tmp/ipykernel_268959/1917952585.py:93: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specif

46


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/tmp/ipykernel_268959/1917952585.py:93: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specif

,corpusID,character,animacy,coref_chain,chain_head,head_of_head,chain_len,CL,SS,NER,WN,DP,TP,CN,TF
0,story1,1,1,"[ dragon , he , he , the dragon , He , hi...",dragon,dragon,43,2.499540,1.0,0.0,0.0,1.0,1.0,1.0,0.022546
1,story1,1,1,"[ princess , the tsar 's daughter , her , h...",princess,princess,23,0.990763,1.0,0.0,1.0,1.0,1.0,1.0,0.007958
2,story1,1,1,"[ tsar , tsar , father , tsar , her father...",tsar,tsar,9,-0.065380,1.0,0.0,1.0,1.0,1.0,1.0,0.006631
3,story1,0,1,"[ princess' dog , a little dog that had follo...",princess' dog,dog,4,-0.442574,1.0,0.0,0.0,1.0,1.0,1.0,0.003979
4,story1,1,1,"[ tsarina , mother , tsarina , tsarina ]",tsarina,tsarina,4,-0.442574,1.0,0.0,1.0,1.0,1.0,1.0,0.002653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907,story46,0,1,"[ the entire enemy army , the enemy army , h...",the entire enemy army,army,7,-0.308434,0.0,0.0,0.0,0.0,0.0,0.0,0.003775
1908,story46,0,1,"[ this , the best solution ]",this,this,2,-0.604613,1.0,0.0,0.0,1.0,0.0,0.0,0.001258
1909,story46,0,1,"[ a horse , a horse ]",a horse,horse,2,-0.604613,0.0,0.0,0.0,0.0,0.0,0.0,0.002517
1910,story46,0,1,"[ an even better horse , his horse , his hor...",an even better horse,horse,4,-0.486142,0.0,0.0,0.0,0.0,0.0,0.0,0.002517


Importing Features from Intermediate File

In [1]:
#import libraries

import os
import sys
import numpy as np
import torchtext
import pandas as pd
import re

from lib import data_utils, preprocess

#get path
src_path = os.getcwd()

datapath = src_path + '/Data/'
PLcopath = datapath + 'AnnotatedPLData/PLCoref'
PLpath = datapath + 'AnnotatedPLData/PLTexts'
CENcopath = datapath + 'AnnotatedCENData/CENCoref'
CENpath = datapath + 'AnnotatedCENData/CENCoref'
ONpath = datapath + 'AnnotatedONData/ONCoref'

PLInter = src_path + '/output/IntermediateFilesPL/'
CENInter = src_path + '/output/IntermediateFilesCEN/'
ONInter = src_path + '/output/IntermediateFilesON/'

#import data

#create empty data frame
PL_Int_data = pd.DataFrame(columns=['corpusID', 'character', 'animacy', 'coref_chain', 'chain_head', 'head_of_head', 'chain_len', 'CL'])

#append all PL texts and features into one dataframe
for n in range(1,47):
# for n in range(1,2):

    print(n)

    #get story path
    storycopath = PLcopath +'/story' + str(n) + '.txt'
    storypath = PLpath + '/story' + str(n) + '.txt'
    storyid = 'story'+ str(n)

    #read in story
    corpus = data_utils.read_story(storycopath)

    #read in from intermediate files
    #list of features
    features = ["CN", "Dep", "NER", "SS", "Triple", "WN"]

    for f in features:
        #empty list
        feat = []
        with open(PLInter + f + 'FeatureBoolean'+'/Story' + str(n) + '.txt', 'r') as doc:
            for line in doc:
                feat.append(eval(line.rstrip()))
    
        corpus[f] = feat
    
    #creat feature for term freq
    tf_dict = preprocess.term_freq(storypath)
    corpus['TF'] = corpus['head_of_head'].map(tf_dict)
    
    #append to dataframe
    PL_Int_data = pd.concat([PL_Int_data, corpus], ignore_index=True)
PL_Int_data

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


,corpusID,character,animacy,coref_chain,chain_head,head_of_head,chain_len,CL,CN,Dep,NER,SS,Triple,WN,TF
0,story1,1,1,"[ dragon , he , he , the dragon , He , hi...",dragon,dragon,43,2.499540,1.0,1.0,0.0,1.0,1.0,0.0,0.022546
1,story1,1,1,"[ princess , the tsar 's daughter , her , h...",princess,princess,23,0.990763,1.0,1.0,0.0,1.0,0.0,1.0,0.007958
2,story1,1,1,"[ tsar , tsar , father , tsar , her father...",tsar,tsar,9,-0.065380,1.0,1.0,0.0,1.0,1.0,1.0,0.006631
3,story1,0,1,"[ princess' dog , a little dog that had follo...",princess' dog,dog,4,-0.442574,1.0,1.0,0.0,1.0,1.0,0.0,0.003979
4,story1,1,1,"[ tsarina , mother , tsarina , tsarina ]",tsarina,tsarina,4,-0.442574,1.0,1.0,0.0,1.0,0.0,1.0,0.002653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907,story46,0,1,"[ the entire enemy army , the enemy army , h...",the entire enemy army,army,7,-0.308434,0.0,1.0,0.0,0.0,0.0,0.0,0.003775
1908,story46,0,1,"[ this , the best solution ]",this,this,2,-0.604613,0.0,0.0,0.0,1.0,0.0,0.0,0.001258
1909,story46,0,1,"[ a horse , a horse ]",a horse,horse,2,-0.604613,0.0,0.0,0.0,0.0,1.0,0.0,0.002517
1910,story46,0,1,"[ an even better horse , his horse , his hor...",an even better horse,horse,4,-0.486142,0.0,0.0,0.0,0.0,1.0,0.0,0.002517


Model

In [2]:
# simple model with their features

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
from joblib import dump, load

data_y = PL_Int_data["character"].astype('int')
data_x = PL_Int_data[["CL", "CN", "Dep", "NER", "SS", "Triple", "WN", "animacy"]]

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, random_state=1)

rbf_svc = svm.SVC(kernel = 'rbf', C=.5, gamma=1)

#fit
rbf_svc.fit(X_train, y_train)

#save
dump(rbf_svc, "models/PLjahan_model.joblib")
rbf_svc = load('models/PLjahan_model.joblib') 

cv_results = cross_validate(rbf_svc, X_train, y_train, cv = 10, scoring=('f1', 'accuracy'), return_train_score=True)

print(cv_results['train_accuracy'])
print(cv_results['test_accuracy'])
print(cv_results['train_f1'])
print(cv_results['test_f1'])

print("%0.2f accuracy with a standard deviation of %0.2f" % (cv_results['test_accuracy'].mean(), cv_results['test_accuracy'].std()))
print("%0.2f f1 score with a standard deviation of %0.2f" % (cv_results['test_f1'].mean(), cv_results['test_f1'].std()))
# cv = StratifiedShuffleSplit(n_splits=10, test_size=.2, random_state=42)

# C_range = np.logspace(-2,10,13)
# gamma_range = np.logspace(-9,3,13)

# grid = GridSearchCV(svm.SVC(kernel='rbf'), param_grid=dict(gamma = gamma_range, C= C_range), cv=cv)

# grid.fit(X_train,y_train)

# print(grid.best_params_, grid.best_score_)



[0.90542636 0.91162791 0.90775194 0.91550388 0.90627421 0.90859799
 0.91092177 0.90472502 0.91247095 0.91092177]
[0.92361111 0.88194444 0.91666667 0.86111111 0.94405594 0.91608392
 0.87412587 0.91608392 0.84615385 0.90909091]
[0.820059   0.83381924 0.82474227 0.83899557 0.82284041 0.82697947
 0.83063328 0.82043796 0.83357879 0.83113069]
[0.86075949 0.76712329 0.84615385 0.72972973 0.89473684 0.83333333
 0.75       0.84210526 0.71052632 0.83544304]
0.90 accuracy with a standard deviation of 0.03
0.81 f1 score with a standard deviation of 0.06


In [3]:
# simple model with their features + term freq

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
from joblib import dump, load

data_y = PL_Int_data["character"].astype('int')
data_x = PL_Int_data[["CL", "CN", "Dep", "NER", "SS", "Triple", "WN", "animacy", "TF"]]

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, random_state=1)

rbf_svc_tf = svm.SVC(kernel = 'rbf', C=.5, gamma=1)

#fit
rbf_svc_tf.fit(X_train, y_train)

#save
dump(rbf_svc_tf, "models/PLjahan_model_tf.joblib")
rbf_svc_tf = load('models/PLjahan_model_tf.joblib') 

cv_results = cross_validate(rbf_svc_tf, X_train, y_train, cv = 10, scoring=('f1', 'accuracy'), return_train_score=True)

print(cv_results['train_accuracy'])
print(cv_results['test_accuracy'])
print(cv_results['train_f1'])
print(cv_results['test_f1'])

print("%0.2f accuracy with a standard deviation of %0.2f" % (cv_results['test_accuracy'].mean(), cv_results['test_accuracy'].std()))
print("%0.2f f1 score with a standard deviation of %0.2f" % (cv_results['test_f1'].mean(), cv_results['test_f1'].std()))

[0.90542636 0.91162791 0.90775194 0.91550388 0.90627421 0.90859799
 0.91092177 0.90472502 0.91247095 0.91092177]
[0.92361111 0.88194444 0.91666667 0.86111111 0.94405594 0.91608392
 0.87412587 0.91608392 0.84615385 0.90909091]
[0.820059   0.83381924 0.82474227 0.83899557 0.82284041 0.82697947
 0.83063328 0.82043796 0.83357879 0.83113069]
[0.86075949 0.76712329 0.84615385 0.72972973 0.89473684 0.83333333
 0.75       0.84210526 0.71052632 0.83544304]
0.90 accuracy with a standard deviation of 0.03
0.81 f1 score with a standard deviation of 0.06


In [9]:
# simple model 

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
from joblib import dump, load

PL_data = pd.read_csv('Data/PL.csv')

data_y = PL_data["character"].astype('int')
data_x = PL_data[["CL", "CN", "DP", "NER", "SS", "TP", "WN", "animacy"]]

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, random_state=1)

rbf_svc_1 = svm.SVC(kernel = 'rbf', C=.5, gamma=1)

#fit
rbf_svc_1.fit(X_train, y_train)

#save
dump(rbf_svc_1, "models/PLallen_model.joblib")
rbf_svc_1 = load('models/PLallen_model.joblib') 

cv_results = cross_validate(rbf_svc_1, X_train, y_train, cv = 10, scoring=('f1', 'accuracy'), return_train_score=True)

print(cv_results['train_accuracy'])
print(cv_results['test_accuracy'])
print(cv_results['train_f1'])
print(cv_results['test_f1'])

print("%0.2f accuracy with a standard deviation of %0.2f" % (cv_results['test_accuracy'].mean(), cv_results['test_accuracy'].std()))
print("%0.2f f1 score with a standard deviation of %0.2f" % (cv_results['test_f1'].mean(), cv_results['test_f1'].std()))

[0.87751938 0.87674419 0.8744186  0.88372093 0.87916344 0.87141751
 0.88071263 0.87451588 0.88613478 0.87916344]
[0.88194444 0.875      0.90277778 0.80555556 0.88811189 0.86013986
 0.81818182 0.90909091 0.8041958  0.83916084]
[0.7641791  0.76720351 0.75892857 0.78323699 0.76785714 0.75149701
 0.77151335 0.76176471 0.77894737 0.77058824]
[0.79012346 0.75       0.82051282 0.64102564 0.78378378 0.6969697
 0.64864865 0.82666667 0.6        0.70886076]
0.86 accuracy with a standard deviation of 0.04
0.73 f1 score with a standard deviation of 0.08


In [11]:
# simple model 

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

data_y = PL_data["character"].astype('int')
data_x = PL_data[["CL", "CN", "DP", "NER", "SS", "TP", "WN", "animacy", "TF"]]

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, random_state=1)

rbf_svc_1tf = svm.SVC(kernel = 'rbf', C=.5, gamma=1)

#fit
rbf_svc_1tf.fit(X_train, y_train)

#save
dump(rbf_svc_1tf, "models/PLallen_model_tf.joblib")
rbf_svc_1tf = load('models/PLallen_model_tf.joblib') 

cv_results = cross_validate(rbf_svc_1tf, X_train, y_train, cv = 10, scoring=('f1', 'accuracy'), return_train_score=True)

print(cv_results['train_accuracy'])
print(cv_results['test_accuracy'])
print(cv_results['train_f1'])
print(cv_results['test_f1'])

print("%0.2f accuracy with a standard deviation of %0.2f" % (cv_results['test_accuracy'].mean(), cv_results['test_accuracy'].std()))
print("%0.2f f1 score with a standard deviation of %0.2f" % (cv_results['test_f1'].mean(), cv_results['test_f1'].std()))

[0.87751938 0.87674419 0.8744186  0.88372093 0.87916344 0.87141751
 0.88071263 0.87451588 0.88613478 0.87916344]
[0.88194444 0.875      0.90277778 0.80555556 0.88811189 0.86013986
 0.81818182 0.90909091 0.8041958  0.83916084]
[0.7641791  0.76720351 0.75892857 0.78323699 0.76785714 0.75149701
 0.77151335 0.76176471 0.77894737 0.77058824]
[0.79012346 0.75       0.82051282 0.64102564 0.78378378 0.6969697
 0.64864865 0.82666667 0.6        0.70886076]
0.86 accuracy with a standard deviation of 0.04
0.73 f1 score with a standard deviation of 0.08


In [24]:
PL_data.to_csv("Data/PL.csv")
PL_Int_data.to_csv("Data/PLInter.csv")